In [1]:
!pip install tensorflow

In [2]:
!pip install tensorflow-hub

In [3]:
!pip install opencv-python

In [4]:
!pip install numpy

In [5]:
!pip install matplotlib

In [6]:
!pip install --upgrade tensorflow-hub

In [7]:
import sys
print(sys.executable)  # Shows exact python path running the notebook
!{sys.executable} -m pip show tensorflow-hub

C:\Users\santhosh\anaconda3\envs\tfenv\python.exe
Name: tensorflow-hub
Version: 0.16.1
Summary: TensorFlow Hub is a library to foster the publication, discovery, and consumption of reusable parts of machine learning models.
Home-page: https://github.com/tensorflow/hub
Author: Google LLC
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\santhosh\appdata\roaming\python\python310\site-packages
Requires: numpy, protobuf, tf-keras
Required-by: 


In [8]:
!pip install pillow ipywidgets ipython

In [1]:
!pip install scikit-image

  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
    --------------------------------------- 0.3/12.8 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.8 MB 1.0 MB/s eta 0:00:12
   -- ------------------------------------- 0.8/12.8 MB 1.2 MB/s eta 0:00:11
   --- ------------------------------------ 1.0/12.8 MB 1.2 MB/s eta 0:00:10
   --- ------------------------------------ 1.0/12.8 MB 1.2 MB/s eta 0:00:10
   ---- ----------------------------------- 1.3/12.8 MB 1.0 MB/s eta 0:00:11
   ---- ----------------------------------- 1.6/12.8 MB 1.1 MB/s eta 0:00:11
   ------ --------------------------------- 2.1/12.8 MB 1.1 MB/s eta 0:00:10
   ------- -------------------------------- 2.4/12.8 MB 1.1 MB/s eta 0:00:10
   -------- ------------------------------- 2.6/12.8 MB 1.2 MB/s eta 0:00:09
   --------- -------------------

In [2]:
# Importing all the required packages and libraries
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display
from skimage.color import rgb2lab, lab2rgb
from skimage.exposure import adjust_gamma

# Upload widget
upload = widgets.FileUpload(accept='image/*', multiple=False)
display(upload)

FileUpload(value=(), accept='image/*', description='Upload')

In [5]:

# Brightness adjustment slider
brightness_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Brightness:',
    continuous_update=False
)
display(brightness_slider)

def get_uploaded_image(upload_widget):
    """Process uploaded image and return PIL and numpy array versions"""
    if not upload_widget.value:
        print("❌ No image uploaded.")
        return None, None
    
    try:
        file_info = upload_widget.value[0] if isinstance(upload_widget.value, tuple) else next(iter(upload_widget.value.values()))
        image_data = file_info['content']
        pil_img = Image.open(io.BytesIO(image_data)).convert('RGB')
        return pil_img, np.array(pil_img)
    except Exception as e:
        print(f"❌ Error processing image: {e}")
        return None, None

def preprocess_for_esrgan(image_array):
    """Prepare image for ESRGAN model (divisible by 4)"""
    h, w = image_array.shape[:2]
    new_h, new_w = (h // 4) * 4, (w // 4) * 4
    cropped = image_array[:new_h, :new_w]
    return cropped.astype(np.float32) / 255.0

def enhance_resolution(model, original_img, brightness=1.0):
    """
    True resolution enhancement with brightness control:
    1. Process original through ESRGAN
    2. Blend with original colors in LAB space
    3. Adjust brightness while preserving details
    """
    # Preprocess original image
    original_processed = preprocess_for_esrgan(original_img)
    original_lab = rgb2lab(original_processed)
    
    # Apply super-resolution
    sr_output = model(tf.convert_to_tensor(original_processed[np.newaxis, ...]))
    sr_rgb = tf.squeeze(sr_output).numpy()
    sr_rgb = np.clip(sr_rgb, 0, 1)
    sr_lab = rgb2lab(sr_rgb)
    
    # Get dimensions
    sr_h, sr_w = sr_rgb.shape[:2]
    
    # Resize original LAB to match SR dimensions (for color channels)
    original_lab_resized = resize(original_lab, (sr_h, sr_w), 
                                 order=1, mode='reflect', anti_aliasing=True)
    
    # Adjust brightness in L channel (0-100 range)
    enhanced_l = np.clip(sr_lab[:,:,0] * brightness, 0, 100)
    
    # Combine enhanced L with original AB channels
    blended_lab = np.zeros_like(sr_lab)
    blended_lab[:,:,0] = enhanced_l
    blended_lab[:,:,1:] = original_lab_resized[:,:,1:]
    
    # Convert back to RGB
    final_rgb = lab2rgb(blended_lab) * 255.0
    final_rgb = np.clip(final_rgb, 0, 255).astype(np.uint8)
    
    # Apply gamma correction if brightness was adjusted
    if brightness != 1.0:
        final_rgb = adjust_gamma(final_rgb, gamma=1.0/brightness)
    
    return final_rgb

def plot_comparison(original, enhanced):
    """Show resolution comparison with zoomed details"""
    plt.figure(figsize=(20, 10))
    
    # Original image (resized to match enhanced dimensions)
    plt.subplot(1, 2, 1)
    resized_original = np.array(Image.fromarray(original).resize(
        (enhanced.shape[1], enhanced.shape[0]), Image.BICUBIC))
    plt.imshow(resized_original)
    plt.title(f"Original (resized to {enhanced.shape[1]}×{enhanced.shape[0]})")
    plt.axis('off')
    
    # Enhanced image
    plt.subplot(1, 2, 2)
    plt.imshow(enhanced)
    plt.title(f"Enhanced ({enhanced.shape[1]}×{enhanced.shape[0]})")
    plt.axis('off')
    
    # Zoomed comparison (center region)
    plt.figure(figsize=(20, 5))
    zoom_size = min(200, enhanced.shape[0]//3, enhanced.shape[1]//3)  # 1/3 of smaller dimension or 200px
    y, x = enhanced.shape[0]//2 - zoom_size//2, enhanced.shape[1]//2 - zoom_size//2
    
    plt.subplot(1, 2, 1)
    plt.imshow(resized_original[y:y+zoom_size, x:x+zoom_size])
    plt.title("Original (Zoomed)")
    plt.axis('off')
    
    plt.subplot(1, 2, 2)
    plt.imshow(enhanced[y:y+zoom_size, x:x+zoom_size])
    plt.title("Enhanced (Zoomed)")
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Main processing
if upload.value:
    # Load model
    print("⏳ Loading ESRGAN model...")
    model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
    print("✅ Model loaded!")
    
    # Get uploaded image
    pil_img, img_array = get_uploaded_image(upload)
    
    if img_array is not None:
        print(f"📊 Original dimensions: {img_array.shape}")
        
        # Apply resolution enhancement
        print("⚙️ Enhancing image resolution...")
        enhanced_img = enhance_resolution(model, img_array, brightness_slider.value)
        
        # Display comparison
        plot_comparison(img_array, enhanced_img)
        
        # Save option
        display(Image.fromarray(enhanced_img))
    else:
        print("❌ Failed to process image.")
else:
    print("ℹ️ Please upload an image.")

# Interactive brightness adjustment
def on_brightness_change(change):
    if upload.value and 'img_array' in globals():
        print("🔄 Updating with new brightness...")
        enhanced_img = enhance_resolution(model, img_array, change['new'])
        display(Image.fromarray(enhanced_img))

brightness_slider.observe(on_brightness_change, names='value')

FloatSlider(value=1.0, continuous_update=False, description='Brightness:', max=2.0, min=0.5)

⏳ Loading ESRGAN model...



✅ Model loaded!
📊 Original dimensions: (415, 410, 3)
⚙️ Enhancing image resolution...


NameError: name 'resize' is not defined